In [1]:
import sys
sys.path.append('../input/timmmaster')
sys.path.append("../input/tez-lib")

In [2]:
import tez
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import math

In [3]:
class args:
    batch_size = 16
    image_size = 384

In [4]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# Dataset

In [5]:
class PawpularDataset:
    def __init__(self, image_paths, dense_features, targets, augmentations):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        targets = self.targets[item]
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }


In [6]:
class PawpularModel(tez.Model):
    def __init__(self,model_name):
        super().__init__()
        self.model=timm.create_model(model_name,pretrained=False,in_chans=3)
        self.model.head=nn.Linear(self.model.head.in_features,128)
        self.dropout=nn.Dropout(0.5)#もともとは、ここが0.1だった
        self.dense1=nn.Linear(140,64)
        self.dense2=nn.Linear(64,1)
    def forward(self,image,features,targets=None):
        x=self.model(image)
        x=self.dropout(x)
        x=torch.cat([x,features],dim=1)
        x=self.dense1(x)
        x=self.dense2(x)
        return x,0,{}
    

In [7]:
test_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [8]:
super_final_predictions=[]
for fold_ in range(10):
    #Intialize the model
    model = PawpularModel(model_name="swin_large_patch4_window12_384")
    
    #load model from path
    model.load(f"../input/paw-models/model_f{fold_}.bin", device="cuda", weights_only=True)
    
    #load test.csv file
    df_test=pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    #create list of path+image ids from the test csv Ids values
    test_img_paths=[f"../input/petfinder-pawpularity-score/test/{id_}.jpg" for id_ in df_test['Id'].values]
    
    #create list of dense features
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
        
    #Create dataset
    test_dataset=PawpularDataset(
        image_paths=test_img_paths,
        dense_features=df_test[dense_features].values,
        targets=np.ones(len(test_img_paths)),
        augmentations=test_aug)
    
    #run data through model
    test_predictions=model.predict(test_dataset,batch_size=2*args.batch_size,n_jobs=-1)
        
        
    
    #put predictions in final_test_predictions list
    final_test_predictions=[]
    for preds in tqdm(test_predictions):
        final_test_predictions.extend(preds.ravel().tolist())
        
    #apply sigmoid and multiply by 100 to final_test_predsictions list 
    final_test_predictions=[sigmoid(x)*100 for x in final_test_predictions]
    super_final_predictions.append(final_test_predictions)
        
        
#add the list to super final predictions list
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)        
#take average over all the folds


# Mycode

In [9]:
df_test['Pawpularity']=super_final_predictions
df_test=df_test[['Id','Pawpularity']]
df_test
df_test.to_csv("submission.csv", index=False)   
df_test